# Visualization

Let's load first an image


In [ ]:
import tifffile

filename = '../data/example.tif'
img = tifffile.imread(filename)
print("The shape of the array is [depth x channels x height x width]", img.shape)

We can display a 2D image using imshow from matplotlib. To display the two channels together, we create a RGB image by coding a small function 'to_rgb' than takes as an input an image with 2 channels with channel being the first index and return an images with 3 channels with channels as last index.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def to_rgb(array):
    ''' Convert the a cxy 2 channel image to a xyc rgb image'''
    shp = (array.shape[1],array.shape[2],3)
    rgb = np.zeros(shp,dtype=array.dtype)
    rgb[:,:,0] = array[0,:,:]
    rgb[:,:,1] = array[1,:,:]
    return rgb

plt.imshow(to_rgb(img[12,:,:,:]))
plt.axis('off')
plt.show()

We can install a scale bar to the figure using the scalebar module that can be installed using ```pip install matplotlib-scalebar```. We'll also see how to save the figure to a pdf file that can be later used for publication.

In [ ]:
# let's load the module
from matplotlib_scalebar.scalebar import ScaleBar

fig, ax = plt.subplots()
plt.imshow(to_rgb(img[12,:,:,:]))
scalebar = ScaleBar(0.08, "um", length_fraction=0.25, color=[1,1,1],box_color=[0,0,0],location='lower right')
ax.add_artist(scalebar)
plt.axis('off')
plt.savefig('../scratch/figure.png',dpi=600)
plt.show()

## Maximum intensity projection
A usual to visualize 3D data sets is to compute a maximum intensity projection.

In [ ]:
import numpy as np

print('The original image shape is', img.shape)

# compute the maximum intensity projection along the 1st axis (index 0)
mip = np.amax(img,axis=0)

# display the result
plt.imshow(to_rgb(mip))
plt.axis('off')
plt.title('Maximum intensity projection')
plt.show()

Let's add a widget to explore the z serie. We'll reuse here the image already loaded and the function to_rgb to convert the image plane by plane. We need to code an update function that refresh the displayed image and used it in the inteact function which will display a widget slider controling the z plane.

In [ ]:
from ipywidgets import *

slider = widgets.IntSlider(value=1,min=1,max=25,step=1,description="Z",disabled=False)

def update(z):
    if z < img.shape[0]:
        plt.imshow(to_rgb(img[z-1,:,:,:]))
        plt.axis('off')
        plt.title(f"Slice: {z}")

interact(update,z=slider);



## Cropping


In [ ]:
import tifffile
import matplotlib.pyplot as plt

img = tifffile.imread('../data/nuclei.tif')

# To crop the image, we use the index of the rows and columns we want to extract 
# from the image using the syntax img[row1:row2,column1:column2]
crop = img[0:200,0:500]
plt.imshow(crop)
plt.title('Cropped image')
plt.axis('off')
plt.show()

## Histogram
Histograms provide a quick insight on the distribion of the intensity in the image.

In [ ]:
import matplotlib.pyplot as plt
from skimage.exposure import histogram

filename = '../data/nuclei.tif'
img = tifffile.imread(filename)
hist, hist_centers  = histogram(img)

fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].imshow(img, cmap=plt.cm.gray)
axes[0].axis('off')
axes[1].plot(hist_centers, hist, lw=2)
axes[1].set_title('histogram of gray values')
fig.show()

## 3D rendering of volumes
Often images acquired in microsopcy are tri-dimensional and we need a way to visualize them in their original full dimensionnality.

### Using ipyvolume (deprecated section)
We can use ipyvolume to render interactively images in a notebook. This will run only in a browser. We install ipyvolume in conda using: 
```conda install -c conda-forge ipyvolume```

In [10]:
import ipyvolume as ipv
import tifffile
# Load a 3D image
filename = '../data/example.tif'
img = tifffile.imread(filename)
# render it in 3D
ipv.figure()
ipv.volshow(img[:,1,:,:])

ModuleNotFoundError: No module named 'ipyvolume'

### Using napari

Napari is a visualization program designed for browsing, annotating, and analyzing large multi-dimensional images.
```conda install -c conda-forge napari ``` (you may need to select a specific version of networkx ```conda install -c conda-forge networkx=2.5```).


We first visualize our volume with its 2 channels:

In [6]:
import tifffile
import napari

# Load a 3D image
filename = '../data/example.tif'
img = tifffile.imread(filename)

# Create a Viewer and add an image here with the two channels and scaling
viewer = napari.view_image(img, channel_axis=1, name=['dapi', 'fitc'], scale=[10,1,1])

/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Let's add an iso-surface. We first smooth the surface using a Gaussian filter and then compute a triangulation of the isosurface using the marching cubes algorithm.

In [9]:
import numpy as np
from skimage import measure
from skimage import filters
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

iso_val = 64 # iso value / threshold

# prefilter the image to produce a smoother surface
vol = filters.gaussian(np.squeeze(img[:,1,:,:]),2,preserve_range=True)

# Create a surface at an isovalue (or threshold) using the marching cube algorithm
verts, faces, normals, values = measure.marching_cubes_lewiner(vol, iso_val)

# The surface is defined by a set of vertices (x,y,z coordinates) and the list faces (triplet of vertices indices). We optionally here have a values array
surface = (verts, faces, values)

# create a Viewer and add an image here with the two channels and scaling
viewer = napari.view_image(img, channel_axis=1, name=['dapi', 'fitc'], scale=[10,1,1])
viewer.add_surface(surface,colormap='jet',scale=[10,1,1])

<ipython-input-9-88b35e0aaf32>:13: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  verts, faces, normals, values = measure.marching_cubes_lewiner(vol, iso_val)
/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)
/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/napari/_vispy/vispy_surface_layer.py:91: UserWarning: Alternative shading modes are only available in 3D, defaulting to none
  warnings.warn(


<Surface layer 'surface' at 0x7fd35fb86490>

Napari can load chunks of image from a zarr file store in order to enable the visualization of large data sets (for example a few ~TB). Let's use the file created in the first example.

In [ ]:
# import the zarr module
import zarr

# Open a zarr file
z = zarr.open('../scratch/tmp.zarr', mode='r')

# Display the file as if it was the original numpy image:
viewer = napari.view_image(z, channel_axis=1, name=['dapi', 'fitc'], scale=[10,1,1])
